In [1]:
import numpy as np
from sklearn.externals import joblib


In [2]:
def giveSuggestions(features,model,numberOfSuggestions=3):
    predictProbabilities=model.predict_proba(features) 
    indeces=predictProbabilities[0].argsort()[-numberOfSuggestions:][::-1]
    #print("Suggestions in decresing of their priorities")
    suggest=[]
    for i in range(len(indeces)):
        #print(i," : ",model.classes_[indeces[i]])
        suggest.append(model.classes_[indeces[i]])
    
    return suggest

In [3]:
import pandas as pd
data=pd.read_csv('FoodCrops.csv')
districtCode=data.iloc[:,1]
districtCodeJson=districtCode.to_json()
paddyCols = [col for col in data.columns if 'PD_Y' in col]
wheatCols = [col for col in data.columns if 'WT_Y' in col]
maizeCols = [col for col in data.columns if 'MZ_Y' in col]
milletCols = [col for col in data.columns if 'ML_Y' in col]
barleyCols = [col for col in data.columns if 'BL_Y' in col]
#districtCodeJson

In [4]:
#!pip install flask
#!pip install -U flask-cors

In [5]:
from flask import Flask, jsonify, request, render_template
from flask_cors import CORS
app = Flask(__name__)
CORS(app)

In [6]:
@app.route('/predict', methods=['POST'])
def predictroute():
    print(request.is_json)
    content=request.get_json()
    ph = content['ph']
    rainfall=content['rainfall']
    temp=content['temp']
    alt=content['alt']
    #print("ph is ",ph)
    bestCrop=giveSuggestions([[ph,temp,alt,rainfall]],lrmodel,5)
    #bestCrop= lrmodel.predict([[ph,temp,alt,rainfall]])
    #print(bestCrop)
    #query_df = pd.DataFrame(json_)
    #query = pd.get_dummies(query_df)
    #prediction = clf.predict(query)
    #return jsonify({'prediction': list(prediction)})
    return jsonify({'prediction': bestCrop})

In [7]:
@app.route('/',methods=['GET'])
def index():
    return render_template('maps.html')

In [8]:
@app.route('/stats',methods=['GET'])
def stats():
    return render_template('stats.html',dc=districtCodeJson)

In [9]:
@app.route('/stats/<code>',methods=['GET'])
def getstats(code):
    id= int(code) #type conversion from string to int
    return jsonify({
        'Paddy':[{'x':paddyCols,'y':data[paddyCols].iloc[id].values.tolist()}],
        'Maize':[{'x':maizeCols,'y':data[maizeCols].iloc[id].values.tolist()}],
        'Millet':[{'x':milletCols,'y':data[milletCols].iloc[id].values.tolist()}],
        'Barley':[{'x':barleyCols,'y':data[barleyCols].iloc[id].values.tolist()}],
        'Wheat':[{'x':wheatCols,'y':data[wheatCols].iloc[id].values.tolist()}] 
    })

In [10]:
if __name__ == '__main__':
     lrmodel=joblib.load('model.joblib')
     app.run(port=8080)

D:\SOFTWARE\Anaconda\lib\site-packages\sklearn\base.py:311: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.2 when using version 0.19.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


UnsupportedOperation: not writable

In [ ]:
#!pip install statsmodels